# Efficient Data Stream Anomaly Detection

Python script capable of detecting anomalies in a continuous data stream. This stream, simulating real-time sequences of floating numbers, could represent various metrics such as financial transaction or system metrics. Focus will be on identifying unusual patterns, such as exceptionally high values or deviation from the norm.

In [1]:
!git init

Initialized empty Git repository in D:/anaconda3/envs/ml_projects/ipnyb-src/.git/


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint:
hint: 	git config --global init.defaultBranch <name>
hint:
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint:
hint: 	git branch -m <name>


In [2]:
!git remote add origin https://github.com/haziq2010443/Data-Stream-Anomaly-Detection.git